In [1]:
import re
import sys
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)

In [2]:
sys.path.append('../..')

In [3]:
from ingestion.ingestion_mediator import IngestionMediator
from scrapers.scraper import Scraper
from utils.utils import SOURCES
mediator = IngestionMediator()

/home/johnny/jk-apps/bike-price-predictor/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
# Combine all raw csv files into agg df so can analyze bike types and subtypes
def aggregate_raw(specs=False):
    """Aggregate raw data files for either products or specifications."""
    if specs:
        tablename = 'product_specs'
    else:
        tablename = 'products'
    manifest_rows = mediator.get_rows_matching(sources=SOURCES, tablenames=[tablename])
    for i, row in enumerate(manifest_rows):
        csv_path = mediator.get_filepath_for_manifest_row(row)
        if i == 0:
            agg_df = pd.read_csv(csv_path)
            continue
        tmp_df = pd.read_csv(csv_path)
        agg_df = agg_df.append(other=tmp_df, ignore_index=True, sort=True)
    return agg_df

# confirms all sites included in df
agg_df = aggregate_raw()
print('Are there sites unique to df or SOURCEs:')
set(agg_df.site.unique()) ^ set(SOURCES)

Are there sites unique to df or SOURCEs:


set()

In [5]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7506 entries, 0 to 7505
Data columns (total 10 columns):
bike_type      7506 non-null object
brand          7506 non-null object
description    7506 non-null object
href           7506 non-null object
model_year     338 non-null float64
msrp           7506 non-null float64
price          7498 non-null float64
product_id     7506 non-null object
site           7506 non-null object
subtype        7506 non-null object
dtypes: float64(3), object(7)
memory usage: 586.5+ KB


In [6]:
# Note: gravity subtype seems to be imply full-suspension and might actually be a brand...
# Also 'frames' doesn't imply actual frames just aggregate category
norm_df = agg_df.copy()
# normalize subtype values, replace '"' with '_inch', remove "complete"
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(Scraper.normalize_spec_fieldnames)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: x.replace('"', '_inch').replace('complete_', ''))
# mountain bike subtypes
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'dirt_jump' if 'dirt_jump' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'dirt_jump' if 'park_dirt' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'cross_country' if 'cross_country' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'cross_country' if 'xc' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'downhill' if 'downhill' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fat' if 'fat' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'trail' if 'trail' in x and 'xc' not in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'hardtail' if 'hardtail' in x and 'xc' not in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'hardtail' if 'hard_tail' in x and 'xc' not in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'enduro' if 'enduro' in x else x)
# road bike subtypes
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'triathlon' if 'triathlon' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'time_trail' if 'time_trial' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'endurance' if 'endurance' in x and 'gravel' not in x and 'race' not in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'endurance' if 'gran_fondo' in x or 'fondo' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'racing' if 'performance' in x and 'gravel' not in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'racing' if 'racing' in x or 'race_bikes' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'racing' if 'race' in x and 'gr' not in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'racing' if 'elite_road' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'racing' if 'road_sport_riding' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'active' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'track' if 'track' in x else x)
# adventure bike subtypes
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'gravel' if 'gravel' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'gravel' if 'all_road' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'gravel' if 'gr300' in x or 'gr_race' in x or 'pro_gr' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'cyclocross' if 'cyclocross' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'touring' if 'touring' in x else x)
# city/urban bike subtypes
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fixie' if 'single_speed' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fixie' if 'fixed' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'fitness' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'active' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'leisure' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'recreation' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'allroad' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'fitness' if 'onroad' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'cruiser' if 'cruiser' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'comfort' if 'comfort' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'commuter' if 'commuter' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'hybrid' if 'hybrid' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'hybrid' if 'dual_sport' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'commuter' if 'multi_speed' in x else x)
# standardize bike type similar values used as subtype
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'adventure' if 'adventure' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'road' if 'road' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'mountain' if 'mountain' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'mountain' if 'mtb' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'specialty' if 'utility' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'city_urban' if 'city' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'city_urban' if 'urban' in x else x)
# # specialty bike subtypes
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'cargo' if 'cargo' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'folding' if 'folding' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'tandem' if 'tandem' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'electric' if 'ebike' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'electric' if 'e_bike' in x else x)
# collapse mountain bikes wheel sizes
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: '27_5_inch' if '650b' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: '26_inch' if '26_inch' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: '29ers' if '29_inch' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: '29ers' if '29_mountain' in x else x)
# collapse mountain bikes by suspension type
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'front_suspension' if 'front_suspension' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'full_suspension' if 'full_suspension' in x else x)
norm_df.loc[:, 'subtype'] = norm_df.subtype.apply(lambda x: 'rigid' if 'rigid' in x else x)
norm_df.subtype.value_counts().sort_index()

26_inch                  22
27_5_inch               455
27_5_inch_plus_wheel    256
29ers                   739
adventure                49
cargo                     7
city_urban               19
comfort                 153
commuter                347
cross_country           234
cruiser                 212
cyclocross              203
dirt_jump                21
downhill                 59
electric                181
endurance                98
enduro                  231
fat                      46
fitness                 232
fixie                    36
folding                  21
full_suspension          81
gravel                  673
gravity                  21
hardtail                255
hybrid                  519
mountain                370
other                     3
racing                  785
rigid                    69
road                    338
specialty                 1
tandem                    2
time_trail                9
touring                  90
track               

In [7]:
# Defined main categories: mountain, road,

# Note: electric bikes will not be treated as unique feature since it spans both main and subtypes.
# Will use subtype to map to main bike type category

# collapse these to fewer main categories
# mountain bike category
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'mountain' if 'mountain' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'mountain' if 'dirt_jump' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'mountain' if 'fat' in x else x)
# road bike category
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'road' if 'road' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'road' if 'time_trial' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'road' if 'triathlon' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'road' if 'track' in x else x)
# adventure bike category
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'adventure' if 'adventure' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'adventure' if 'cyclocross' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'adventure' if 'gravel' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'adventure' if 'touring' in x else x)
# city/urban bike category
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'urban' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'hybrid' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'comfort' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'cruiser' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'fitness' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'fixed' in x else x)
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'city_urban' if 'path_pavement' in x else x)
# specialty bike category
norm_df.loc[:, 'bike_type'] = norm_df.bike_type.apply(lambda x: 'specialty' if 'other' in x else x)
norm_df.bike_type.value_counts().sort_index()

adventure      376
city_urban    1512
ebike            3
ebikes           6
electric        32
mountain      3443
road          1973
specialty      161
Name: bike_type, dtype: int64

In [8]:
# dictionary of bike type and its subtypes
bike_type_to_subtypes = {
    'road': ['racing', 'ultralight', 'endurance', 'time_trail', 'triathlon', 'track'],
    'adventure': ['gravel', 'cyclocross', 'touring', 'bikepacking'],
    'city_urban': ['hybrid', 'fitness', 'cruiser', 'comfort', 'commuter', 'fixie',
                  'singlespeed'],
    'specialty': ['cargo', 'folding', 'tandem', 'recumbent', 'tricycles',
                 'unicycles', 'electric'],
    'mountain': ['trail', 'cross_country', 'fat', 'enduro', 'downhill', 'dirt_jump']
}
# map subtypes to bike types
def get_subtypes_to_biketyps(include_biketypes=True):
    subtypes_to_biketypes = dict()
    for bike_type, subtypes in bike_type_to_subtypes.items():
        if include_biketypes:
            subtypes_to_biketypes[bike_type] = bike_type
        for subtype in subtypes:
            subtypes_to_biketypes[subtype] = bike_type
    return subtypes_to_biketypes
subtypes_to_biketypes = get_subtypes_to_biketyps()
subtypes_to_biketypes

{'road': 'road',
 'racing': 'road',
 'ultralight': 'road',
 'endurance': 'road',
 'time_trail': 'road',
 'triathlon': 'road',
 'track': 'road',
 'adventure': 'adventure',
 'gravel': 'adventure',
 'cyclocross': 'adventure',
 'touring': 'adventure',
 'bikepacking': 'adventure',
 'city_urban': 'city_urban',
 'hybrid': 'city_urban',
 'fitness': 'city_urban',
 'cruiser': 'city_urban',
 'comfort': 'city_urban',
 'commuter': 'city_urban',
 'fixie': 'city_urban',
 'singlespeed': 'city_urban',
 'specialty': 'specialty',
 'cargo': 'specialty',
 'folding': 'specialty',
 'tandem': 'specialty',
 'recumbent': 'specialty',
 'tricycles': 'specialty',
 'unicycles': 'specialty',
 'electric': 'specialty',
 'mountain': 'mountain',
 'trail': 'mountain',
 'cross_country': 'mountain',
 'fat': 'mountain',
 'enduro': 'mountain',
 'downhill': 'mountain',
 'dirt_jump': 'mountain'}

In [9]:
# map subtypes to correct bike types
# Note: we are left with bike_type values as subtypes remaining that needs remapping and updating
# for example: adventure, road, etc.
for subtype, bike_type in subtypes_to_biketypes.items():
    norm_df.loc[norm_df.subtype == subtype, 'bike_type'] = bike_type

In [10]:
grouped = norm_df.groupby(by=['bike_type', 'subtype'])
grouped['site'].count()

bike_type   subtype             
adventure   adventure                49
            cyclocross              203
            gravel                  673
            touring                  90
city_urban  city_urban               19
            comfort                 153
            commuter                347
            cruiser                 212
            fitness                 232
            fixie                    36
            hybrid                  519
mountain    26_inch                  22
            27_5_inch               455
            27_5_inch_plus_wheel    256
            29ers                   739
            cross_country           234
            dirt_jump                21
            downhill                 59
            enduro                  231
            fat                      46
            full_suspension          81
            gravity                  21
            hardtail                255
            mountain                370
       

In [11]:
# show updated bike type counts
norm_df.bike_type.value_counts().sort_index()

adventure     1015
city_urban    1518
mountain      3418
road          1335
specialty      220
Name: bike_type, dtype: int64

In [12]:
# Note: looks like some observations come from different sites are actually duplicates
# see: agg_df.loc[norm_df.subtype == 'other',]
# TODO: figure out how to actually remove such dupes...in thise case excluding 'site' column
# all other columns are exactly the same, hence we should treat as duplicates
agg_df.loc[norm_df.subtype == 'other',]

,bike_type,brand,description,href,model_year,msrp,price,product_id,site,subtype
1812,other,Surly,Surly Big Fat Dummy - 2019,/product/surly-big-fat-dummy-328513-1.htm,NaN,3025.0,3025.0,328513,proshop,other
2829,other,Surly,Surly Big Fat Dummy - 2019,/product/surly-big-fat-dummy-328513-1.htm,NaN,3025.0,3025.0,328513,contebikes,other
4394,other,Surly,Surly Big Fat Dummy - 2019,/product/surly-big-fat-dummy-328513-1.htm,NaN,3025.0,3025.0,328513,spokes,other


## Summary

### Main categories
#### Mountain bikes
These are designed for riding rough off-road trails. They typically have flat or upright handlebars, and a very low gear range for pedaling up steep trails.

Mountain bike subtypes are:
- trail bikes: This is arguably the most common mountain biking style because the category isn’t grounded in any specific type of racing.
- cross-country bikes: This style of riding typically implies riding fast, with an emphasis on climbing prowess. Distances vary from just a few miles to 25-plus, and bikes tend to focus on efficiency and low weight. These bikes can be great if you’re considering getting competitive or would like a racier ride for your local trails.
- fat bikes: Oversize tires, from 3.7 in. to 5+ in. wide, give these bikes excellent traction, especially in sand or snow. Fat bikes are great for beginners because the wide tires are reassuringly forgiving as a rider picks a line through rough terrain.
- all-mountain/enduro bikes: Think of all-mountain riding as trail riding on steroids, with bigger leg-burning climbs, longer, scarier descents and more technical features—both man-made and natural. Bikes for all-mountain riding are designed to perform well on steep descents while also being light and nimble enough to pedal uphill.
- downhill/park/freeride bikes: Mostly ridden at lift-serviced bike parks. Downhill bikes are big and tough, and riders wear full-face helmets and body armor as they encounter jumps, berms, rock gardens and wooden ladders.
- dirt jump: these bikes are a cross breed. This time though they are a fusion between Freeride and BMX bikes. They also sometimes known as urban, or street mountain bikes. They tend to only have suspension in the front. Put simple, Dirt Jumpers are the bike best suited for the riders who like to spend their time in the air doing aerial stunts. Many of these bikes will have single speed gears, only one brake, oversized handlebars, small frames, and low seat posts for stunt riding. They will also be ridden by people with no fear.

In addition, suspension type and wheel diameter are two key attributes that determine the type of terrain a bike is capable of handling. You’ll also want to consider things like frame material, number of gears and type of brakes as you narrow down your bike choice.

#### Road bikes
These are made for speed, with narrow tires for rolling fast and far on paved surfaces. They have smooth, skinny tires and "drop" handlebars, and can be used for on-road racing. They are usually lighter than other types of bicycles.

They can be ridden on paved trails, but most people find them uncomfortable and unstable on unpaved trails. Most road bikes are not capable of carrying heavy loads, so are not very suitable for commuting or touring. Disc-brake versions are becoming increasingly common. Many brands have women’s-specific models, which have different touch points (saddle, handlebars, and sometimes crankarm length) for women’s anatomy and sometimes have women’s-specific frame geometry.

Road bike subtypes are:
- aero/racing: With wing-shape frame tubes and wheels, these bikes reduce drag to a bare minimum. Intended for racing or time-trialing, they often feature more aggressive rider positions; the emphasis is speed over rider comfort.
- ultralight: Made for long climbs, these bikes strip weight by using the lightest materials, but may not be as good for all-around use.
- endurance/sportive/performance: The right pick for most riders, these models feature a more upright rider position for comfort, slightly wider tires and gearing ranges and, sometimes, vibration-absorbing technologies in the frame and/or fork.
- triathlon/time_trial: A time trial is a discipline of road bike racing (also a part of triathlons) where riders leave the starting line one by one and ride alone along the course. It’s the famous race against the clock. Whoever completes the course in the shortest time wins. In part because drafting (or riding in another rider’s slipstream) is forbidden, TT bikes are purpose-built to offer the absolute least wind resistance possible.
- track: These bikes are made for riding or racing on a track, also known in cycling as a velodrome. Because they’re made for these racing ovals, and based on the types of racing done on tracks, these bikes are, effectively, the stripped-down cousins of racing bikes. Stripped-down the point of not having brakes, even, on many models.

There's quite a bit of overlap between ultralight and aero/racing bikes - aero/racing bikes typically also tend to be light weight bikes as their price increases.

#### Adventure bikes
These somewhat tricky to categorise but essentially, with a classic drop handlebar, a gravel/adventure bike looks almost exactly like a traditional road bike. But look closer, and you’ll find differences like clearance for much larger, knobby tires for off-road use, additional mounts for accessories like bottle cages and frame bags, and a geometry that’s designed for a more upright rider position for comfort and stable handling on unpaved surfaces. Virtually all of these bikes today have disc brakes and wide-range gearing for steep climbs. Some incorporate front or rear suspension elements for rider comfort. More companies are making women’s-specific models, which typically differ from men’s in parts selection, like saddle model and handlebar width.

Adventure bikes subtypes are:
- gravel: An all-around, all-surface bike made for a range of uses, from races to all-day adventures.
- cyclocross: A stripped-down version of an adventure bike for racing. Tire clearance is similar, but geometry features sharper, more aggressive handling, and there are often no additional accessory mounts beyond two bottle cages.
- touring/bikepacking: The extreme adventure version, bikepacking bikes have clearance for fat, 29x2-inch tires, lots of mounts for accessories and, sometimes, suspension-corrected geometry so you can outfit it with a suspension fork.

These go-anywhere, ride-anything machines are the most versatile bikes on the market today. They combine much of the fast-rolling efficiency of a road bike with treaded tires for traction off pavement. They can handle almost any kind of riding you want to do with the exception of road races. Some models have rack and fender eyelets, making them serviceable tourers and commuters.

#### City/Urban bikes
If it’s a bike built for fun—but not a road or mountain bike—it’s probably in this broad category that encompasses everything from flat-bar fitness bikes to beach cruisers. A hybrid/city/comfort road bike is a cross between a road bike and a mountain bike, sharing features of both while more resembling a road bike. It has a lightweight, often aluminum, frame like a road bike but has a more upright geometry and setup, featuring a flat, higher handlebar like a mountain bike, with a larger “comfort” seat and oftentimes easier gearing. It also has ample clearance for tires wider than a normal road bike, and most often has mounts for racks and fenders, and cantilever or disc brakes. If you picture the types of bikes many Europeans ride daily as transportation, you’ve got a good idea of what a hybrid looks like.

City/urban subtypes:
- hybrid: Named because they share features of road and mountain bikes, these feature the upright rider position, broad-range gearing and flat handlebar of a mountain bike, with narrower, fast-rolling tires with light tread. Some have suspension.
- fitness: hese emphasize road bike-like efficiency without the aggressive position, with lightweight frames, performance parts, and narrow slick tires paired to flat handlebars for a more comfortable rider position.
- comfort: These bikes are marked by wide, lightly-treaded tires, cushioned seats (and sometimes suspension) and a very upright rider position.
- cruiser: The classic swoopy-framed beach cruiser with fat tires, 1-7 gears and wide, chopper-style handlebars.
- commuter: Efficient and fast-rolling, this bike typically has flat handlebars for an upright position that lets you see well in urban environments. Multi-gear drivetrains handle hills, rim or disc brakes stop safely, and high-end versions come with accessories like fenders, racks and even lights pre-installed.
- fixie: Stylish and slick, these are the café racers of urban bikes. They’re based on track racing bikes, with lightweight frames, only one gear, and a clean, stripped-down aesthetic, often with color-matched parts and accessories.


#### Specialty bikes
These bikes defy easy classification: They are essentially any bike made with utility, rather than recreation, in mind. They come in various, purpose-built styles. Whatever you pick, you get exercise, stuff gets done, and you always get free parking. You’ll find traditional “diamond” frame and step-through models in all styles except fixed gear (fixie) for anyone who wants an easier way to get on the bike.

Specialty bike subtypes:
- cargo: There are two styles: long-tail and front-load (Dutch: bakfiets). Long-tail is a stretched rear wheelbase with built-in racks and pannier compatibility for large loads. Front-load uses a cargo bin ahead of the pilot and between the wheels. The low center of gravity works great for heavy hauling.
- folding: Foldable bikes are bicycles which can be folded into a compact size and easily transported on trains, buses, and airplanes. This also allows riders to store them or carry them around easily. Foldable bikes use various folding mechanisms, some of which are better than others.
- recumbent: A recumbent bike is a bicycle that allows the rider to be in a laid-back position while cycling. This position allows for the weight of the rider to be better distributed, so it reduces low back pain and other problems.
- tandem: Simply, a bike made to carry two riders. To handle the extra weight and long wheelbase, the frames are built from oversized tubing, and often feature a second downtube that bisects the bike to add stiffness. Crucial parts like wheels and tires are typically built for strength and durability as well. Pay special attention to the brakes; you’ll want powerful versions for sure, safe stopping (discs are best). Some production manufacturers make tandems, but they’re also popular items from custom framebuilders.

### Custom columns
#### Electric
Electric bikes is a broad category that covers all main bike types and their subtypes. So makes the most sense to treat this as it's own unique attribute. So as part of processing bike types and subtypes a method will be generated that identifies whether the data point is an electric bike or not (categorical attribute).
#### Suspension types
There are three standard suspension types:
- rigid: Not the most common type of mountain bike, “rigid” mountain bikes don’t feature any suspension. They are easy to maintain and usually less expensive, but most riders prefer bikes with suspension for greater comfort. Most fat bikes are rigid because riders find that the wide tires and low tire pressure provide all the squish needed to absorb bumps on the trail.
- hardtail: These bikes have a suspension fork in the front to help absorb impact on the front wheel, but the rear of the bike has no suspension—ergo a hardtail. Hardtails are typically less expensive than full-suspension bikes and have fewer moving parts (which often translates into less maintenance). Most hardtails have the ability to lock out the front fork for times when a fully rigid bike is desired.
- full-suspension: There are many variations of full-suspension bikes, but the general idea is for the front fork and rear shock to absorb the impacts of the trail. This drastically reduces the impact on the rider, increases traction, and makes for a more forgiving and enjoyable ride.

A full-suspension bike can soak up a lot of a trail bumps and chatter, but the bike can also “bob” a bit and you lose some of the energy transfer when climbing uphill. As a result, most full-suspension rigs have the ability to lock-out the rear suspension to offer better power transfer and more efficient climbing.
#### Mountain bike wheel size
There are several wheel size options availabl for mountain bikes:
- 26 in.: In the not too-distant past, all adult mountain bikes were equipped with 26 in. wheels. It still is an available wheel size, but now when you walk into a bike shop and inquire about mountain bikes, you are likely to be asked, “26 in., 27.5 in. or 29 in.?”
- 27.5 in. (650b): Offering a middle ground between standard 26 in. wheels and 29ers, these bikes apply a “best of both worlds” solution, more easily rolling over terrain than the 26s, but more maneuverable than 29ers. 27.5 in. wheels can be found on both full-suspension and hardtail rigs.
- 29ers: These bikes feature 29 in. wheels that are a little slower to accelerate, but once you start moving you can conquer considerably more terrain far easier than on a bike with 26 in. wheels. They are more efficient for longer rides as they keep their momentum up and they have a higher “attack angle,” meaning the wheel rolls over trail obstacles easier. These bikes have become extremely popular for the cross-country crowd. 29ers can be found in rigid, hardtail and full-suspension rigs.
- 27.5+ in.: The plus symbol simply indicates extra-wide wheels and tires, typically 2.8 in. or more in width. Wider tires offer a more comfortable and more forgiving ride. They also encounter less rolling resistance, so the trend is for bikes to have wider wheels and tires these days.
- 24 in: Kids’ mountain bikes typically have 24 in. wheels to accommodate the shorter legs of children. Most are less-expensive versions of adult bikes with simpler components. Generally speaking, these suit kids ages 10 to 13, but this depends more on the size of the child than the age. Younger/smaller children can get started on mountain bikes with 20 in. wheels.

## Clean up Bike Types and Subtypes
There are still incorrect bike type to subtype mapping. For some, we still have bike_type values mapped as subtypes. Need to clean this up by looking at additional metadata. For example, we can use the description field to help identify the real subtypes for those with invalid subtype values. Additionally, some specifications data include "recommended use" and "details" fields that might be useful in maximizing resolution.

### First pass with 'desc' field

In [28]:
valid_subtypes = set(get_subtypes_to_biketyps(include_biketypes=False).keys()) - {'electric'}
valid_subtypes

{'bikepacking',
 'cargo',
 'comfort',
 'commuter',
 'cross_country',
 'cruiser',
 'cyclocross',
 'dirt_jump',
 'downhill',
 'endurance',
 'enduro',
 'fat',
 'fitness',
 'fixie',
 'folding',
 'gravel',
 'hybrid',
 'racing',
 'recumbent',
 'singlespeed',
 'tandem',
 'time_trail',
 'touring',
 'track',
 'trail',
 'triathlon',
 'tricycles',
 'ultralight',
 'unicycles'}

In [25]:
invalid_subtypes = set(norm_df.subtype.unique()) - valid_subtypes

{'26_inch',
 '27_5_inch',
 '27_5_inch_plus_wheel',
 '29ers',
 'adventure',
 'city_urban',
 'electric',
 'full_suspension',
 'gravity',
 'hardtail',
 'mountain',
 'rigid',
 'road',
 'specialty'}

In [15]:
for i in norm_df.loc[norm_df.subtype.isin(invalid_subtypes),].index:
    for valid in valid_subtypes:
        desc = norm_df.description[i].lower()
        # look for whole word cases of valid
        regex = r'\b' + re.escape(valid) + r'\b'
        if re.search(regex, desc, re.IGNORECASE):
            norm_df.loc[i, 'subtype'] = valid
            continue

In [16]:
grouped = norm_df.groupby(by=['bike_type', 'subtype'])
grouped['site'].count()

bike_type   subtype             
adventure   adventure                36
            commuter                  1
            cyclocross              203
            gravel                  685
            touring                  90
city_urban  city_urban               18
            comfort                 154
            commuter                347
            cruiser                 212
            fitness                 232
            fixie                    36
            hybrid                  519
mountain    26_inch                  22
            27_5_inch               421
            27_5_inch_plus_wheel    239
            29ers                   698
            cross_country           234
            dirt_jump                21
            downhill                 60
            enduro                  264
            fat                      67
            full_suspension          70
            gravity                  21
            hardtail                242
       

### Merge with Specifications

In [29]:
specs_df = aggregate_raw(specs=True)
specs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7506 entries, 0 to 7505
Columns: 404 entries, 0_0_mm to wiring_harness
dtypes: float64(51), object(353)
memory usage: 23.1+ MB


In [30]:
# Confirm have at least 'product_id' column to merge prods_df on specs_df
print('Same number of rows:', len(norm_df) == len(specs_df))
common_cols = set(norm_df.columns) & set(specs_df.columns)
print('Intersecting columns:', common_cols)

# Confirm same number of product ids - symmetric difference i.e. unique to each should be zero
set(norm_df.product_id.unique()) ^ set(specs_df.product_id.unique())

Same number of rows: True
Intersecting columns: {'site', 'model_year', 'product_id'}


set()

In [31]:
# Merge on common columns using only subset of specs_df
mod_specs_df = specs_df[['site', 'product_id', 'model_year', 
                         'recommended_use', 'best_use', 'intended_use', 'details']]
merged_df = norm_df.merge(right=mod_specs_df, how='outer', on=list(common_cols))
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8214 entries, 0 to 8213
Data columns (total 14 columns):
bike_type          7506 non-null object
brand              7506 non-null object
description        7506 non-null object
href               7506 non-null object
model_year         708 non-null float64
msrp               7506 non-null float64
price              7498 non-null float64
product_id         8214 non-null object
site               8214 non-null object
subtype            7506 non-null object
recommended_use    897 non-null object
best_use           276 non-null object
intended_use       194 non-null object
details            7501 non-null object
dtypes: float64(3), object(11)
memory usage: 962.6+ KB


In [32]:
# let's take look at unique values for recommended_use
merged_df.loc[merged_df.recommended_use.notna(), 'recommended_use'].value_counts()
# Note: will need to will need to split by column and count each unique subtype value

trail                                170
road cycling                         158
enduro, trail                        158
enduro                                97
gravel                                72
cross-country                         58
cross-country, trail                  49
cyclocross                            28
cyclocross, gravel                    19
cross-country, trail, bikepacking     10
enduro, freeride                       9
cross-country, enduro, trail           8
time trial, triathlon                  8
downhill                               6
downhill, enduro, trail                6
enduro, freeride, downhill             6
cycling                                5
downhill, enduro, freeride             4
trail, enduro                          4
gravel, bikepacking                    3
mud/sand/snow                          3
downhill, enduro                       2
freeride, enduro                       2
road endurance                         2
triathlon       

In [33]:
# let's take look at unique values for best_use
merged_df.loc[merged_df.best_use.notna(), 'best_use'].value_counts()
# Note: will need to will need to split by column and count each unique subtype value

Mountain Biking                       91
Recreational Cycling                  59
Urban Cycling                         42
Road Cycling                          38
Cycling                               15
Bike Touring                           8
Cyclocross                             6
Fitness                                4
Bike Touring, Bikepacking              3
Bikepacking, Road Cycling              3
Road Cycling, Bikepacking              2
Road Cycling, Recreational Cycling     1
Bike Commuting                         1
Bikepacking                            1
Mountain Biking, Bikepacking           1
Bikepacking, Mountain Biking           1
Name: best_use, dtype: int64

In [34]:
# let's take look at unique values for intended_use
merged_df.loc[merged_df.intended_use.notna(), 'intended_use'].value_counts()
# Note: will need to will need to split by column and count each unique subtype value

Trail, All-Mountain                      13
Trail, All-mountain                      10
Road                                      8
Gravel, Cyclocross                        8
Trail                                     8
Enduro, Trail                             7
Trail, All-mountain, Enduro               7
All-mountain, Trail                       7
Trail, All-Mountain, Enduro               6
XC                                        6
Trail, Cross-Country                      6
XC, Trail                                 5
Road, Commuting, Adventure, Endurance     5
Enduro, All-Mountain                      5
Cross-Country, Light Trail                4
Road, Cyclocross, Gravel                  4
Trail, Enduro                             4
Enduro, Park, All-mountain                4
Enduro, All mountain, Park, Trail         4
Trail, Enduro, All-Mountain               4
Touring, Bikepacking, Commuting           3
Elite Road, Endurance                     3
Race                            

In [40]:
# let's get a sense of what subtype values we can filter from 'description' field
# and other bike_type/subtype similar fields like 'recommended use', 'best_use', 'intended_use'.
# Then use 'details' field for more data points.
potential_subtypes = dict()

for invalid in invalid_subtypes:
        idx = merged_df.loc[merged_df.subtype == invalid].index
        index_dict = dict()
        for i in idx:
#             desc = str(merged_df.loc[i, 'description']).lower()
            rec_use = str(merged_df.loc[i, 'recommended_use']).lower()
            best_use = str(merged_df.loc[i, 'best_use']).lower()
            intended_use = str(merged_df.loc[i, 'intended_use']).lower()
            details = str(merged_df.loc[i, 'details']).lower()
            counter_dict = dict()
            for subtype in valid_subtypes:
                count = counter_dict.get(subtype, 0)
#                 count = count + 1 if subtype in desc else count
                count = count + 1 if subtype in rec_use else count
                count = count + 1 if subtype in best_use else count
                count = count + 1 if subtype in intended_use else count
                count += 1 if subtype in details else count
                if count > 0:
                    counter_dict[subtype] = count
            if len(counter_dict) > 0:
                index_dict[i] = counter_dict
        potential_subtypes[invalid] = index_dict
potential_subtypes

# Note: seeing some useful patterns that might allow for generic mapping for some of these
# invalid subtypes. This should be used as part of EDA for model building not defaul cleaner
# process...requires looking at each observations and identifying the appropriate subtype.
# For example, full_suspension tends to be mapped to 'trail' but sometimes also included
# other subtypes as options.
# road types sometimes have a counts like: 4: {'comfort': 1, 'bikepacking': 1, 'gravel': 1, 'touring': 1}
# this should be mapped to 'touring' since bikepacking and touring are essentiall the same.
# Also seem to have some data integrity issues? Have road with 'trail' subtype??? Will need to look
# closer at these observations and determine validity.

{'full_suspension': {1608: {'track': 1, 'trail': 1, 'downhill': 1},
  1610: {'trail': 1},
  1611: {'trail': 1},
  1612: {'trail': 1},
  1614: {'trail': 1},
  1615: {'trail': 1},
  1645: {'comfort': 1, 'trail': 1},
  1675: {'enduro': 1, 'trail': 1},
  3411: {'track': 1},
  3412: {'track': 1},
  3417: {'trail': 1},
  3418: {'trail': 1},
  3721: {'track': 1, 'trail': 1},
  3773: {'trail': 1, 'downhill': 1},
  3774: {'trail': 1, 'downhill': 1},
  3779: {'downhill': 1},
  3785: {'downhill': 1, 'racing': 1},
  4000: {'comfort': 1, 'trail': 1},
  4001: {'comfort': 1, 'trail': 1},
  4002: {'track': 1, 'comfort': 1, 'trail': 1},
  6728: {'track': 1, 'trail': 1},
  6734: {'trail': 1, 'racing': 1},
  6736: {'trail': 1, 'racing': 1},
  6753: {'trail': 1},
  6760: {'downhill': 1},
  6768: {'trail': 1, 'racing': 1},
  6771: {'track': 1, 'trail': 1},
  6772: {'track': 1, 'trail': 1},
  6773: {'trail': 1, 'ultralight': 1, 'racing': 1},
  6774: {'track': 1, 'trail': 1},
  6775: {'track': 1, 'trail': 1}

In [22]:
# let's get a sense of what subtype values we can filter from 'description' field
# and other bike_type/subtype similar fields like 'recommended use', 'best_use', 'intended_use'.
# Then use 'details' field for more data points.
potential_subtypes = dict()

for invalid in invalid_subtypes:
        idx = merged_df.loc[merged_df.subtype == invalid].index
        index_dict = dict()
        for i in idx:
            desc = str(merged_df.loc[i, 'description']).lower()
            counter_dict = dict()
            for subtype in subtypes_to_biketypes.keys():
                count = counter_dict.get(subtype, 0)
                count = count + 1 if subtype in desc else count
                if count > 0:
                    counter_dict[subtype] = count
            if len(counter_dict) > 0:
                index_dict[i] = counter_dict
        potential_subtypes[invalid] = index_dict
potential_subtypes

{'27_5_inch_plus_wheel': {1113: {'fat': 1},
  1447: {'fat': 1},
  1470: {'fat': 1},
  1551: {'fat': 1},
  1569: {'fat': 1},
  1660: {'fat': 1},
  1661: {'fat': 1},
  1666: {'fat': 1},
  1718: {'fat': 1},
  1721: {'fat': 1},
  1731: {'fat': 1},
  1744: {'fat': 1},
  2474: {'fat': 1},
  2475: {'fat': 1},
  2483: {'fat': 1},
  2584: {'fat': 1},
  2622: {'trail': 1},
  2630: {'fat': 1},
  2642: {'fat': 1},
  2650: {'fat': 1},
  2688: {'fat': 1},
  4250: {'fat': 1},
  6942: {'trail': 1}},
 'ebike': {6056: {'road': 1}},
 '29ers': {1115: {'fat': 1},
  1542: {'ultralight': 1},
  1584: {'fat': 1},
  1663: {'fat': 1},
  2415: {'ultralight': 1},
  2472: {'fat': 1},
  2476: {'fat': 1},
  2484: {'fat': 1},
  4282: {'ultralight': 1},
  7424: {'fat': 1},
  7425: {'fat': 1},
  7439: {'fat': 1},
  7440: {'fat': 1}},
 'turbo_e_bike': {},
 'hardtail': {46: {'mountain': 1},
  54: {'mountain': 1},
  55: {'mountain': 1},
  57: {'mountain': 1},
  58: {'adventure': 1, 'mountain': 1},
  59: {'adventure': 1, 'm

In [23]:
# # update subtypes for those with only single subtype option
# for invalid, values in potential_subtypes.items():
#     for index, subtypes in values.items():
#         if len(subtypes) == 1:
#             subtype = list(subtypes.keys())[0]
#             merged_df.loc[index, 'subtype'] = subtype

In [24]:
# for subtype, bike_type in subtypes_to_biketypes.items():
#     merged_df.loc[merged_df.subtype == subtype, 'bike_type'] = bike_type
# grouped = merged_df.groupby(by=['bike_type', 'subtype'])
# grouped['site'].count()

bike_type   subtype             
adventure   adventure                35
            cyclocross              203
            gravel                  692
            touring                  90
city_urban  city_urban               16
            comfort                 154
            commuter                348
            cruiser                 212
            electric                 11
            fitness                 233
            fixie                    36
            hybrid                  523
            singlespeed               1
            turbo_e_bike             15
ebike       ebike                     1
mountain    26_inch                  22
            27_5_inch               417
            27_5_inch_plus_wheel    216
            29ers                   685
            cross_country           234
            dirt_jump                21
            downhill                 60
            e_bike                    1
            electric                  3
       